# Steel Plate Defect prediction


## Preparing train set

In [27]:
import pandas as pd
import numpy as np
from sklearn import model_selection
path = 'input/'

TARGET = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']

if __name__ == "__main__":
    # Training data is in the "train.csv" and "faults.csv" files
    train_data = pd.read_csv(path + 'train.csv')
    original_data = pd.read_csv(path + 'faulty-steel-plates/faults.csv')
    train_data.drop('id', axis=1, inplace=True)
    train_data = pd.concat([train_data, original_data], axis=0)

    # drop mult label data points
    train_data = train_data[train_data[TARGET].sum(axis=1) <= 1]
    train_data['Outside_Global_Index'] = np.where(train_data['Outside_Global_Index']==0.7, 0.5, train_data['Outside_Global_Index'])

    train_data['target'] = np.argmax(train_data[TARGET].values, axis=1) + 1
    train_data.loc[train_data[TARGET].sum(axis=1) == 0, 'target'] = 0
    train_data.drop(TARGET, axis=1, inplace=True)

    # we create a new column called kfold and fill it with -1
    train_data['kfold'] = -1

    # the next step is to randomize the rows of the data
    train_data = train_data.sample(frac=1).reset_index(drop=True)

    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=5)

    # fill the new kfold column
    for f, (t_, v_) in enumerate(kf.split(X=train_data, y=train_data['target'].values)):
        train_data.loc[v_, 'kfold'] = f
    
    # save the new csv with kfold column
    train_data.to_csv(path + 'train_folds.csv', index=False)


    

    